<a href="https://colab.research.google.com/github/wlrma0108/HuggingFace/blob/main/BERT_COMPELET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
!pip install transformers seqeval[gpu]
!pip3 install torch

In [103]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer,BertConfig,BertForTokenClassification
from torch import cuda

In [104]:
device ='cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [105]:
data=pd.read_csv("ner_datasetreference.csv", encoding='unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [106]:
print("Number of tags: {}".format(len(data.Tag.unique())))
frequencies= data.Tag.value_counts()
frequencies

Number of tags: 17


O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: Tag, dtype: int64

In [107]:
tags={}
for tag, count in zip(frequencies.index,frequencies):
  if tag !="0":
    if tag[2:5] not in tags.keys():
      tags[tag[2:5]]=count
    else:
      tags[tag[2:5]] +=count
  continue

print(sorted(tags.items(),key=lambda x:x[1],reverse=True))

[('', 887908), ('geo', 45058), ('org', 36927), ('per', 34241), ('tim', 26861), ('gpe', 16068), ('art', 699), ('eve', 561), ('nat', 252)]


In [108]:
entities_to_remove=["B-art","I-art","B-eve","I-eve","B-nat","I-nat"]
data=data[~data.Tag.isin(entities_to_remove)]
data.head()
data=data.fillna(method='ffill')

In [109]:
data.head()


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [110]:
data['sentence']=data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Word'].transform(lambda x : ' '. join(x))
data['word_labels']=data[['Sentence #', 'Word','Tag']].groupby(['Sentence #'])['Tag'].transform(lambda x : ','.join(x))
data.head()

,Sentence #,Word,POS,Tag,sentence,word_labels
0,Sentence: 1,Thousands,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
1,Sentence: 1,of,IN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
2,Sentence: 1,demonstrators,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
3,Sentence: 1,have,VBP,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
4,Sentence: 1,marched,VBN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."


In [111]:
label2id = {k: v for v, k in enumerate(data.Tag.unique())}
id2label = {v: k for v, k in enumerate(data.Tag.unique())}
label2id


{'B-geo': 1,
 'B-gpe': 2,
 'B-org': 5,
 'B-per': 3,
 'B-tim': 7,
 'I-geo': 4,
 'I-gpe': 9,
 'I-org': 6,
 'I-per': 8,
 'I-tim': 10,
 'O': 0}

In [112]:
data=data[["sentence","word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

,sentence,word_labels
0,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
1,Families of soldiers killed in the conflict jo...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-per,O,O,..."
2,They marched from the Houses of Parliament to ...,"O,O,O,O,O,O,O,O,O,O,O,B-geo,I-geo,O"
3,"Police put the number of marchers at 10,000 wh...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
4,The protest comes on the eve of the annual con...,"O,O,O,O,O,O,O,O,O,O,O,B-geo,O,O,B-org,I-org,O,..."


In [113]:
len(data)

47571

In [114]:
data.iloc[13].sentence

"Two Germans and four Nigerian oil workers were kidnapped by armed militants during a raid on a boat in Nigeria 's southern oil-rich Delta region ."

In [115]:
data.iloc[13].word_labels

'O,B-gpe,O,O,B-gpe,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-geo,O,O,O,B-geo,O,O'

In [116]:
MAX_LEN=128
TRAIN_BATCH_SIZE=4
VALID_BATCH_SIZE=2
EPOCHS=1
LEARNING_RATE=1e-05
MAX_GRAD_NORM=10
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [117]:
def tokenize_and_preserve_labels(sentence, text_labels,tokenizer):
  tokenized_sentence=[]
  labels=[]
  sentence=sentence.strip()
  for word, label in zip(sentence.split(), text_labels.split(",")):
    tokenized_word=tokenizer.tokenize(word)
    n_subwords = len(tokenized_word)
    tokenized_sentence.extend(tokenized_word)
    labels.extend([label]*n_subwords)

  return tokenized_sentence, labels

In [281]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
    self.len=len(dataframe)
    self.data=dataframe
    self.tokenizer=tokenizer
    self.max_len=max_len

  def __getitem__(self, index):
    sentence = self.data.sentence[index]
    word_labels=self.data.word_labels[index]
    tokenized_sentence,labels= tokenize_and_preserve_labels(sentence,word_labels,self.tokenizer)


    tokenized_sentence=["[CLS]"]+tokenized_sentence+["[SEP]"]
    
    labels.insert(0,"O")
    labels.insert(-1,"O")

    maxlen = self.max_len

    if(len(tokenized_sentence)>maxlen):
      tokenized_sentence = tokenized_sentence[:maxlen]
      labels = labels[:maxlen]
    else:
      tokenized_sentence=tokenized_sentence +['[PAD]' for _ in range(maxlen - len(tokenized_sentence))]
      labels=labels +["O" for _ in range(maxlen - len(labels))]

    attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]
    ids=self.tokenizer.convert_tokens_to_ids(tokenized_sentence)




    label_ids = [label2id[label] for label in labels]

    return{
        
        'ids': torch.tensor(ids,dtype=torch.long),
        'mask': torch.tensor(attn_mask,dtype=torch.long),
        'targets': torch.tensor(label_ids,dtype=torch.long)
    }
  def __len__(self):
        return self.len 
 

In [282]:
train_size=0.8
train_dataset=data.sample(frac=train_size,random_state=200)
test_dataset=data.drop(train_dataset.index).reset_index(drop=True)
train_dataset=train_dataset.reset_index(drop=True)

print("Full Dataset : {}".format(data.shape))
print("TRAIN Dataset : {}".format(train_dataset.shape))
print("TEST Dataset : {}".format(test_dataset.shape))

training_set=dataset(train_dataset,tokenizer,MAX_LEN)
testing_set=dataset(test_dataset,tokenizer,MAX_LEN)
training_set[0]



Full Dataset : (47571, 2)
TRAIN Dataset : (38057, 2)
TEST Dataset : (9514, 2)


{'ids': tensor([  101, 23564, 21030,  2099,  4967,  2001,  9388,  1011,  6109,  2005,
          2634,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [283]:
training_set[0]

{'ids': tensor([  101, 23564, 21030,  2099,  4967,  2001,  9388,  1011,  6109,  2005,
          2634,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [285]:
training_set[0]["ids"]

tensor([  101, 23564, 21030,  2099,  4967,  2001,  9388,  1011,  6109,  2005,
         2634,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [286]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]),training_set[0]["targets"][:30]):
  print('{0:10} {1}'.format(token,id2label[label.item()]))

[CLS]      O
za         B-per
##hee      B-per
##r        B-per
khan       I-per
was        O
mar        O
-          O
93         O
for        O
india      B-geo
.          O
[SEP]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O
[PAD]      O


In [287]:
train_params={'batch_size': TRAIN_BATCH_SIZE,
              'shuffle': True,
              'num_workers':0
              }
test_params={'batch_size': VALID_BATCH_SIZE,
             'shuffle':True,
             'num_workers':0}

training_loader=DataLoader(training_set,**train_params)
testing_loader=DataLoader(testing_set, **test_params)

In [288]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', 
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)
model.to(device)

In [291]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss
tr_logits = outputs[1]
tr_logits.shape
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)


In [292]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        flattened_targets = targets.view(-1) 
        active_logits = tr_logits.view(-1, model.num_labels) 
        flattened_predictions = torch.argmax(active_logits, axis=1) 
        active_accuracy = mask.view(-1) == 1
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [293]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.61011004447937
Training loss per 100 training steps: 0.44284193129232613
Training loss per 100 training steps: 0.2896595930226555
Training loss per 100 training steps: 0.22569768099838317
Training loss per 100 training steps: 0.1895103319280983
Training loss per 100 training steps: 0.16531302479324406
Training loss per 100 training steps: 0.14867414615447083
Training loss per 100 training steps: 0.1348818702578077
Training loss per 100 training steps: 0.12511414796702294
Training loss per 100 training steps: 0.11689066618306772
Training loss per 100 training steps: 0.11004742322262603
Training loss per 100 training steps: 0.10378684372481273
Training loss per 100 training steps: 0.0989631358580625
Training loss per 100 training steps: 0.0947462657465099
Training loss per 100 training steps: 0.09078747966392584
Training loss per 100 training steps: 0.08745514593713188
Training loss per 100 training steps: 0.08413442069892048
Trai